## APRENDIZADO DE MÁQUINA NÃO SUPERVISIONADO:

## Redução de dimensionalidade utilizando o algoritmo PCA (Principal Componentes Analysis)

O PCA é um dos principais algoritmos de aprendizagem de máquina não supervisionada.

Ele identifica a correlação entre variáveis e, caso haja forte correlação entre elas, decide que é possível reduzir a dimensionalidade dos dados.

Supondo um número m de variáveis, o PCA extrai um novo número p <= m que explica a melhor variação na base de dados, excluindo a variável dependente y. Tal número m é escolhido pelo projetista.


# Parte 1: Importação de bibliotecas

In [19]:
import pandas as pd

import numpy as np

# Conversão de dados categóricos em numéricos:
from sklearn.preprocessing import LabelEncoder
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
# Sugestão de leitura: Como fazer LabelEncoder em um DataFrame — Python/SKLearn
# https://gianmedeirao.medium.com/como-fazer-labelencoder-em-um-dataframe-python-sklearn-655ba2c6ae7e

# Normalização dos dados
from sklearn.preprocessing import StandardScaler
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
# Sugestão de leitura: How to Use StandardScaler and MinMaxScaler Transforms in Python:
# https://machinelearningmastery.com/standardscaler-and-minmaxscaler-transforms-in-python/
# https://www.hashtagtreinamentos.com/padronizacao-e-normalizacao-em-ciencia-de-dados

from sklearn.model_selection import train_test_split
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
# Sugestão de leitura:
# https://www.geeksforgeeks.org/how-to-do-train-test-split-using-sklearn-in-python/

from sklearn.decomposition import PCA
# https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

# Classificador Random Forest
from sklearn.ensemble import RandomForestClassifier
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

# Pacote para cálculo da acurácia do modelo de classificação:
from sklearn.metrics import accuracy_score
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html

# Parte 2: Preparação dos dados

O conjunto de dados modificado "census.csv" consiste em aproximadamente 32.000 registros de dados, com cada registro de dados tendo 13 características.

https://archive.ics.uci.edu/dataset/20/census+income

Este conjunto de dados é uma versão modificada do conjunto de dados publicado no artigo "*Scaling Up the Accuracy of Naive-Bayes Classifiers: a Decision-Tree Hybrid*", de Ron Kohavi.

Artigo:

https://aaai.org/papers/033-scaling-up-the-accuracy-of-naive-bayes-classifiers-a-decision-tree-hybrid/


In [3]:
# Criando um dataframe a partir do dados do arquivo
df = pd.read_csv('/content/household_power_consumption.csv')
df.head()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.84,18.4,0,1,17.0
1,16/12/2006,17:25:00,5.36,0.436,233.63,23,0,1,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.29,23,0,2,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.74,23,0,1,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.68,15.8,0,1,17.0


In [4]:
df.shape

(1048575, 9)

In [14]:
# Seleção de atributos (Dados de entrada)
# Com o método .iloc selecionamos (por índice) todas as linhas,
# e as colunas de 0 a 13 (o segundo parâmetro é excludente)
X = df.iloc[:, 0:8].values

In [6]:
X

array([['16/12/2006', '17:24:00', '4.216', ..., '18.4', '0', '1'],
       ['16/12/2006', '17:25:00', '5.36', ..., '23', '0', '1'],
       ['16/12/2006', '17:26:00', '5.374', ..., '23', '0', '2'],
       ...,
       ['13/12/2008', '21:36:00', '0.422', ..., '1.8', '0', '0'],
       ['13/12/2008', '21:37:00', '0.422', ..., '1.8', '0', '0'],
       ['13/12/2008', '21:38:00', '0.422', ..., '1.8', '0', '0']],
      dtype=object)

In [7]:
# Seleção do target (Dados de saída)
# Com o método .iloc selecionamos (por índice) todas as linhas, apenas a coluna 14 (income)
y = df.iloc[:, 8].values
# y = df['income'].values

In [8]:
y

array([17., 16., 17., ...,  0.,  0.,  0.])

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 9 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Date                   1048575 non-null  object 
 1   Time                   1048575 non-null  object 
 2   Global_active_power    1048575 non-null  object 
 3   Global_reactive_power  1048575 non-null  object 
 4   Voltage                1048575 non-null  object 
 5   Global_intensity       1048575 non-null  object 
 6   Sub_metering_1         1048575 non-null  object 
 7   Sub_metering_2         1048575 non-null  object 
 8   Sub_metering_3         1044506 non-null  float64
dtypes: float64(1), object(8)
memory usage: 72.0+ MB


In [11]:
# Creating LabelEncoders for each categorical column
le_date = LabelEncoder()
le_time = LabelEncoder()

# Applying Label Encoding to each categorical column
df['Date'] = le_date.fit_transform(df['Date'])
df['Time'] = le_time.fit_transform(df['Time'])

# Displaying the head of the DataFrame to verify the transformation
print(df.head())

   Date  Time Global_active_power Global_reactive_power Voltage  \
0   380  1044               4.216                 0.418  234.84   
1   380  1045                5.36                 0.436  233.63   
2   380  1046               5.374                 0.498  233.29   
3   380  1047               5.388                 0.502  233.74   
4   380  1048               3.666                 0.528  235.68   

  Global_intensity Sub_metering_1 Sub_metering_2  Sub_metering_3  
0             18.4              0              1            17.0  
1               23              0              1            16.0  
2               23              0              2            17.0  
3               23              0              1            17.0  
4             15.8              0              1            17.0  


In [15]:
X

array([[380, 1044, '4.216', ..., '18.4', '0', '1'],
       [380, 1045, '5.36', ..., '23', '0', '1'],
       [380, 1046, '5.374', ..., '23', '0', '2'],
       ...,
       [311, 1296, '0.422', ..., '1.8', '0', '0'],
       [311, 1297, '0.422', ..., '1.8', '0', '0'],
       [311, 1298, '0.422', ..., '1.8', '0', '0']], dtype=object)

In [24]:
# Dropping rows with missing values
df.dropna(inplace=True)

# Separating features (X) and Sub_metering_3 variable (y)
X = df.drop(columns=['Sub_metering_3'])
y = df['Sub_metering_3']

# Assuming '?' is a placeholder for missing values, replacing it with NaN
X.replace('?', pd.NA, inplace=True)

# Converting all columns to numeric
X = X.apply(pd.to_numeric)

# Dropping any remaining rows with missing values after conversion
X.dropna(inplace=True)

# Applying StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [25]:
print(X.dtype)

float64


In [26]:
X

array([[ 0.0813373 ,  0.77981429,  2.75137082, ...,  2.8721802 ,
        -0.18627081, -0.07479918],
       [ 0.0813373 ,  0.78221945,  3.76416779, ...,  3.83783929,
        -0.18627081, -0.07479918],
       [ 0.0813373 ,  0.78462462,  3.77656216, ...,  3.83783929,
        -0.18627081,  0.08261419],
       ...,
       [-0.24700497,  1.38591492, -0.60750304, ..., -0.61258952,
        -0.18627081, -0.23221255],
       [-0.24700497,  1.38832008, -0.60750304, ..., -0.61258952,
        -0.18627081, -0.23221255],
       [-0.24700497,  1.39072524, -0.60750304, ..., -0.61258952,
        -0.18627081, -0.23221255]])

In [27]:
# Com o método train_test_split separamos os dados em treino e teste:
# X_treino e y_treino: dados para treinamento dos modelos (80% dos dados iniciais)
# X_teste e y_teste: dados para teste e avaliação dos modelos (20% dos dados iniciais)

X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.20, random_state=10)

In [28]:
X_treino.shape, X_teste.shape

((835604, 8), (208902, 8))

## Parte 3: Aplicação do algoritmo PCA (Principal component analysis)

In [29]:
# Criamos a variável pca e aplicamos o algoritmo PCA para a redução do número original de colunas para
# um número definido "p"
# O hiperparâmetro "n_components" recebe o valor definido de novas colunas (p)
p = 3
pca = PCA(n_components=p)

In [30]:
# Definimos novas variáveis de treino e testes (agora com um menor números de colunas)
X_treino_pca = pca.fit_transform(X_treino)
X_teste_pca = pca.transform(X_teste)

In [31]:
X_treino.shape, X_teste.shape

((835604, 8), (208902, 8))

In [32]:
X_treino_pca.shape, X_teste_pca.shape

((835604, 3), (208902, 3))

In [33]:
# O método "explained_variance_ratio" permite calcular a
# porcentagem de variância dos dados que consegue ser explicada por cada um dos componentes selecionados

# Relembrando! :-)
# A variância, assim como o desvio padrão, é uma das medidas de dispersão de dados
# que mostra o comportamento dos dados de uma amostra em relação a uma medida central (por exemplo, a média)

pca.explained_variance_ratio_

array([0.37679626, 0.12689024, 0.11798101])

In [34]:
# Somando os percentuais conseguimos ver o quanto nosso
# modelo PCA conseguiu explicar dos dados de entrada
total_variancia = pca.explained_variance_ratio_.sum()
print(f'O modelo PCA com {p} variáveis explica{100*(total_variancia): .2f} % dos dados de entrada')

O modelo PCA com 3 variáveis explica 62.17 % dos dados de entrada


# Parte 4: Aplicando os novos dados gerados em um modelo de classificação.

Utilizaremos o algoritmo de classificação Random Forest com base nos novos dados, após a aplicação da técnica de redução de dimensionalidade.

In [35]:
# Instanciando o objeto do classificador
modelo_rf = RandomForestClassifier(n_estimators=40, random_state=0)

In [36]:
# Usando o método .fit para treinar o modelo

# Com base nos dados de treino iniciais:
# modelo_rf.fit(X_treino, y_treino)

# Com base nos dados de treino gerados após a aplicação do algoritmo PCA:
modelo_rf.fit(X_treino_pca, y_treino)

RandomForestClassifier(n_estimators=40, random_state=0)

In [37]:
# Usando o método .predict para estimar os resultados

# Realizando as previsões com base nos dados de teste iniciais
# previsoes = modelo_rf.predict(X_teste)

# Realizando as previsões com base nos dados de teste após a aplicação do algoritmo PCA
previsoes = modelo_rf.predict(X_teste_pca)


In [38]:
previsoes

array([ 0., 18.,  1., ..., 17., 17.,  0.])

In [39]:
y_teste

989050     0.0
33984     17.0
362567     0.0
326223     0.0
689000     1.0
          ... 
355077    18.0
22758      0.0
720170     0.0
467627    17.0
7026       0.0
Name: Sub_metering_3, Length: 208902, dtype: float64

In [40]:
# Calculando a acurácia do modelo de classificação:
acuracia = accuracy_score(y_teste, previsoes)
print(f'A acurácia do modelo Random Forest é de:{(100*acuracia): .2f} %')

# Sugestão de leitura:
# Métricas de avaliação de modelos de aprendizado de máquina:
# https://mariofilho.com/as-metricas-mais-populares-para-avaliar-modelos-de-machine-learning/

A acurácia do modelo Random Forest é de: 63.18 %
